# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [9]:
########### 1 Evaluating a foundation model ###########################

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorWithPadding, DataCollatorForLanguageModeling
from datasets import load_dataset
import numpy as np

wiki_text = load_dataset("wikitext", "wikitext-2-raw-v1")

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = wiki_text.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

print(tokenized_datasets)

block_size = 128

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
)

print(lm_datasets)

model = AutoModelForCausalLM.from_pretrained('gpt2')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./output/causalLM",
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=1,   
    learning_rate=5e-5,
    num_train_epochs=1,
    save_steps=1000,
    eval_steps=500,
    logging_steps=500,
    evaluation_strategy="steps",
    save_total_limit=2,
    remove_unused_columns=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

results = trainer.evaluate()
perplexity=np.exp(results["eval_loss"]) 
print(f"Perplexity on wikitext:{perplexity.item():.4f}")
print(results)

######################## 2 performing parameter efficient fine-tuneing ##########################

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn"],
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./output/causalLM",
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    learning_rate=5e-5,
    num_train_epochs=1,
    save_steps=1000,
    eval_steps=500,
    logging_steps=500,
    evaluation_strategy="steps",
    save_total_limit=2,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
eval_results = trainer.evaluate()

perplexity=np.exp(eval_results["eval_loss"]) 
print(f"Perplexity on wikitext:{perplexity.item():.4f}")
print(eval_results)

########################## 3 saving model ############################

peft_model.save_pretrained("./output/lora_gpt2_adapters")
tokenizer.save_pretrained("./output/lora_gpt2_adapters") # save the tokenizer 

########################## 4 performing Inference using fine-tuned model #############################

from peft import AutoPeftModelForCausalLM, PeftModel
lora_model_id = "./output/lora_gpt2_adapters"

lora_model = PeftModel.from_pretrained(model, lora_model_id)
lora_model = lora_model.merge_and_unload()
lora_model.config.pad_token_id = lora_model.config.eos_token_id

input_text = "How are you!"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(peft_model.device)

output = lora_model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})
DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2209
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18666
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1931
    })
})


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Perplexity on wikitext:58.0648
{'eval_loss': 4.061559200286865, 'eval_runtime': 34.5542, 'eval_samples_per_second': 63.928, 'eval_steps_per_second': 63.928}


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.23643136409814364


Step,Training Loss,Validation Loss
500,4.048800,3.680179
1000,3.837100,3.613727


Perplexity on wikitext:36.9736
{'eval_loss': 3.6102030277252197, 'eval_runtime': 24.2883, 'eval_samples_per_second': 90.949, 'eval_steps_per_second': 5.723, 'epoch': 1.0}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


How are you!
I'm a big fan of the show and I'm really excited to see what the future holds for the show. I'm excited to see what the future holds for the show and I'm excited to see what the future holds


In [10]:
##################### 1 Evaluating a Fundation Model #################################

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np

splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(5000))

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(tokenize_function, batched=True)
    
model = AutoModelForSequenceClassification.from_pretrained("gpt2",
        num_labels=2,
        id2label={0:"NEGATIVE", 1:"POSITIVE"},
        label2id={"NEGATIVE": 0, "POSITIVE": 1},
        )
model.config.pad_token_id = model.config.eos_token_id

for param in model.parameters():
    param.requires_grad = False
 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metric(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions==labels).mean()}

training_args = TrainingArguments(
        output_dir = "./output/negative_positive",
        evaluation_strategy="epoch",
        per_device_eval_batch_size = 16,
        per_device_train_batch_size = 16,
        num_train_epochs=1,
        learning_rate = 2e-5,
        save_strategy="epoch",
        load_best_model_at_end=True,
        )

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["test"],
        data_collator=data_collator,
        compute_metrics=compute_metric,
        tokenizer=tokenizer,
        )
results = trainer.evaluate() 
print(f"Evaluation results on imdb:")
print(results)


########################## 2 Performing parameter efficient fine-tuning  ###############################

from peft import LoraConfig, get_peft_model, TaskType, PeftModelForSequenceClassification
import torch

config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=32,
        target_modules=['c_attn'],
        lora_dropout = 0.01,
        )

lora_model = PeftModelForSequenceClassification(model, config)
lora_model.config.pad_token_id = lora_model.config.eos_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_model.to(device)

trainer = Trainer(
        model=lora_model,
        args=TrainingArguments(
        output_dir = "./output/negative_positive",
        evaluation_strategy="epoch",
        per_device_eval_batch_size = 1,
        per_device_train_batch_size = 1,
        num_train_epochs=1,
        learning_rate = 2e-5,
        save_strategy="epoch",
        load_best_model_at_end=True,
        ),
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["test"],
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metric,
        tokenizer=tokenizer,
        )
lora_model.print_trainable_parameters()

trainer.train()
eval_results = trainer.evaluate() 
print(f"Evaluation results via fine-tuned model on imdb:")
print(eval_results)

############################## 3 saving model ####################################

lora_model.save_pretrained("./output/my_gpt_lora_sentiment")

############################## 4 performing inference using fine-tuned model #####################################

from peft import AutoPeftModelForSequenceClassification
import torch

my_lora_gpt_model = AutoPeftModelForSequenceClassification.from_pretrained(
                    "./output/my_gpt_lora_sentiment",
                    num_labels=2,
                    id2label={0:"NEGATIVE", 1:"POSITIVE"},
                    label2id={"NEGATIVE": 0, "POSITIVE": 1}
                    )
my_lora_gpt_model.config.pad_token_id = my_lora_gpt_model.config.eos_token_id

my_lora_gpt_model.to(device)

prompt = "this movie is very good."
inputs = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = my_lora_gpt_model(**inputs)
    logits = outputs.logits        

    probabilities = torch.nn.functional.softmax(logits, dim=1)    
    predicted_class_id = probabilities.argmax().item()    
    id2label={0: "NEGATIVE", 1: "POSITIVE"}
    predicted_label = id2label[predicted_class_id]
    
print(predicted_label)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluation results on imdb:
{'eval_loss': 2.1070284843444824, 'eval_accuracy': 0.4986, 'eval_runtime': 417.4162, 'eval_samples_per_second': 11.978, 'eval_steps_per_second': 0.75}


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 297,984 || all params: 124,737,792 || trainable%: 0.23888830740245906


Epoch,Training Loss,Validation Loss,Accuracy
1,1.000800,0.798425,0.868000


Evaluation results via fine-tuned model on imdb:
{'eval_loss': 0.7984251976013184, 'eval_accuracy': 0.868, 'eval_runtime': 448.7467, 'eval_samples_per_second': 11.142, 'eval_steps_per_second': 11.142, 'epoch': 1.0}


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NEGATIVE


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.